NLP problem is sequence to sequence problems

<a href="https://colab.research.google.com/github/Kishore-C-Dev/ML-AI/blob/main/ML/NLP_fundamenetals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-12-26 13:28:04--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-12-26 13:28:04 (82.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import plot_loss_curves , unzip_data , walk_through_dir , create_tensorboard_callback

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2024-12-26 13:28:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-12-26 13:28:20 (97.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
#visualize Text Data
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#Shuffle data
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
import random
random_index=random.randint(0,len(train_df)-1)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples() :
  _,text,target=row
  print(text,"---",target)

The Dress Memes Have Officially Exploded On The Internet http://t.co/yG32yb2jDY --- 0
Hi yall this poem is called is the one about the snowstorm when we meet in space and that one time it rained. Thx. Ur watching disney chann --- 0
Former Freedom Surf/Spa Razed to Make Way for New Homes http://t.co/EX6JzQJ3NI --- 0
quoted here--&gt;CNN: Purported ISIS video threatens Croatian hostage http://t.co/swVuZxi6gT --- 1
Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) - http://t.co/xRP0rTkFfJ --- 1


In [ ]:
#split data into training and validation data
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                      train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [ ]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [ ]:
from os.path import split
#convert text to numbers
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(max_tokens=None,standardize='lower_and_strip_punctuation',split='whitespace',
                                  ngrams=None,output_mode='int',output_sequence_length=None,pad_to_max_tokens=False)

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [ ]:
max_length=15
max_vocab_length=10000
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  output_mode='int',
                                  output_sequence_length=max_length)


In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
text_vectorizer(["There is a flood in my Street"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_scentence = random.choice(train_sentences)
print(f"Original text:\n{random_scentence}\
      \n\nVectorized version:")
text_vectorizer([random_scentence])

Original text:
Massive Typhoon heads toward Taiwan. http://t.co/Na2Ey64Vsg      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1639,  494, 2096, 2254, 1248,    1,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
embedding=Embedding(input_dim=max_vocab_length,output_dim=128)
random_scentence_embedding=embedding(text_vectorizer([random_scentence]))
print(f'original Scentence:  {random_scentence}')
print(f'Embedded Scentence:  {random_scentence_embedding}')
print(f'Embedded Scentence:  {random_scentence_embedding.shape}')

original Scentence:  Massive Typhoon heads toward Taiwan. http://t.co/Na2Ey64Vsg
Embedded Scentence:  [[[ 0.0441504  -0.00551647  0.00466009 ...  0.02104     0.04037276
    0.0005173 ]
  [-0.03743974 -0.02000129  0.04699818 ... -0.02373186 -0.04028574
   -0.03377356]
  [-0.02887806  0.03594399 -0.03851809 ... -0.04230771  0.04109139
    0.02585545]
  ...
  [ 0.04610043  0.04819237 -0.01962526 ...  0.0350917   0.00623288
   -0.01768339]
  [ 0.04610043  0.04819237 -0.01962526 ...  0.0350917   0.00623288
   -0.01768339]
  [ 0.04610043  0.04819237 -0.01962526 ...  0.0350917   0.00623288
   -0.01768339]]]
Embedded Scentence:  (1, 15, 128)


In [ ]:
random_scentence_embedding[0][0],random_scentence_embedding[0][0].shape,random_scentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.0441504 , -0.00551647,  0.00466009, -0.03564862, -0.01274044,
         0.02122969, -0.0289775 , -0.01033777,  0.02059163,  0.01322105,
        -0.01132206, -0.02396692, -0.04210503, -0.0330143 ,  0.03470248,
         0.0324882 ,  0.04279647,  0.00604855,  0.00374106, -0.01335288,
         0.03938932, -0.04349383,  0.01669228,  0.02339132,  0.03062082,
         0.02561465,  0.01461556, -0.04448326, -0.00029287,  0.02437401,
        -0.02620438, -0.04867027, -0.04994422,  0.03970263, -0.04333737,
        -0.01936337, -0.01216089, -0.01653926, -0.04649895, -0.03824761,
         0.00955592, -0.0457961 , -0.03693204,  0.02326076, -0.02002965,
         0.0477489 ,  0.00699519,  0.01488351,  0.03766093,  0.02435483,
         0.03439479, -0.0146009 ,  0.02306885, -0.02904548, -0.00816935,
        -0.00410683,  0.02188993, -0.03567601,  0.04961436, -0.04182569,
        -0.04017855,  0.01671394, -0.03001495, -0.04492738, -0.01335377,
  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clf",MultinomialNB())
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_score=model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [ ]:
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=keras.Model(inputs,outputs,name='model_1_dense')

In [ ]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_1.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

In [ ]:
model_1_history=model_1.fit(x=train_sentences,y=train_labels,epochs=5,
                            validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6482 - loss: 0.6480 - val_accuracy: 0.7598 - val_loss: 0.5316
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8171 - loss: 0.4514 - val_accuracy: 0.7808 - val_loss: 0.4782
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8583 - loss: 0.3556 - val_accuracy: 0.7900 - val_loss: 0.4577
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8927 - loss: 0.2895 - val_accuracy: 0.7900 - val_loss: 0.4633
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9157 - loss: 0.2357 - val_accuracy: 0.7874 - val_loss: 0.4904


In [ ]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7782 - loss: 0.5251


[0.49035561084747314, 0.787401556968689]

In [ ]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


(762, 1)

In [ ]:
model_1_pred_probs=tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [ ]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights.shape

(10000, 128)

In [ ]:
#code from https://www.tensorflow.org/text/guide/word_embeddings
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [ ]:
#code to download files
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#RNN's are useful for sequence problems
#LSTM - Long Short term Memory
# input(text) -> Tokenize -> Embed -> Layers (Rnn's/Dense) -> output (lable property)
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x=layers.LSTM(64,return_sequences=True)(x)
print(x.shape)
x=layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
print(x.shape)
model_2 = keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)
(None, 64)


In [ ]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 15, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,362,497 (5.20 MB)

 Trainable params: 1,362,497 (5.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
mode_2_history=model_2.fit(train_sentences,
          train_labels,
          epochs=5,
          validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9135 - loss: 0.2795 - val_accuracy: 0.7822 - val_loss: 0.5878
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9398 - loss: 0.1566 - val_accuracy: 0.7743 - val_loss: 0.6906
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9489 - loss: 0.1324 - val_accuracy: 0.7769 - val_loss: 0.6751
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9630 - loss: 0.0965 - val_accuracy: 0.7743 - val_loss: 0.8829
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9678 - loss: 0.0814 - val_accuracy: 0.7730 - val_loss: 1.0967


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


((762, 1),
 array([[1.3230281e-03],
        [6.0607773e-01],
        [9.9989355e-01],
        [3.8324147e-03],
        [2.5491868e-04],
        [9.9977189e-01],
        [9.1455901e-01],
        [9.9990296e-01],
        [9.9988091e-01],
        [1.1496319e-01]], dtype=float32))

In [ ]:
model_2_pred_probs = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x=layers.GRU(64,return_sequences=True)(x)
#x=layers.LSTM(64,return_sequences=True)(x)
x=layers.GRU(64)(x)
#x=layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
mode_3_history=model_3.fit(train_sentences,
          train_labels,
          epochs=5,
          validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8655 - loss: 0.2904 - val_accuracy: 0.7782 - val_loss: 0.6647
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9725 - loss: 0.0765 - val_accuracy: 0.7769 - val_loss: 0.7236
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9727 - loss: 0.0732 - val_accuracy: 0.7717 - val_loss: 0.9917
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9756 - loss: 0.0613 - val_accuracy: 0.7782 - val_loss: 1.0789
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9792 - loss: 0.0478 - val_accuracy: 0.7651 - val_loss: 1.1301


In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x=layers.GRU(64,return_sequences=True)(x)
#x=layers.LSTM(64,return_sequences=True)(x)
x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.GRU(64))(x)
#x=layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = keras.Model(inputs, outputs, name="model_4_bidirectional")

In [ ]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 15, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,453,441 (5.54 MB)

 Trainable params: 1,453,441 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
mode_4_history=model_4.fit(train_sentences,
          train_labels,
          epochs=5,
          validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9796 - loss: 0.0362 - val_accuracy: 0.7638 - val_loss: 1.7013
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9851 - loss: 0.0294 - val_accuracy: 0.7769 - val_loss: 1.6186
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9818 - loss: 0.0331 - val_accuracy: 0.7730 - val_loss: 1.6387
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9868 - loss: 0.0266 - val_accuracy: 0.7677 - val_loss: 1.9497
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9857 - loss: 0.0307 - val_accuracy: 0.7703 - val_loss: 1.6210


In [ ]:
embedding_test=embedding(text_vectorizer(["This is a test sentence"]))
conv1d=layers.Conv1D(filters=32,kernel_size=5,activation='relu',padding='valid')
conv1d_output=conv1d(embedding_test)
max_pool=layers.GlobalAveragePooling1D()
max_pool_output=max_pool(conv1d_output)
embedding_test.shape,conv1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
# Convolution Neural Network's for text (sequence problems)
#its Conv2D for images we need to use Conv1D for text
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,activation='relu')(x)
x=layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = keras.Model(inputs, outputs, name="model_5_Conv1D")

In [ ]:
model_5.summary()

Model: "model_5_Conv1D"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_5_history=model_5.fit(train_sentences,
          train_labels,
          epochs=5,
          validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9808 - loss: 0.0427 - val_accuracy: 0.7572 - val_loss: 1.2197
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9833 - loss: 0.0391 - val_accuracy: 0.7598 - val_loss: 1.2264
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9832 - loss: 0.0382 - val_accuracy: 0.7598 - val_loss: 1.3477
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9823 - loss: 0.0395 - val_accuracy: 0.7454 - val_loss: 1.2576
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9844 - loss: 0.0348 - val_accuracy: 0.7480 - val_loss: 1.2605


In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])




tf.Tensor(
[-0.03133019 -0.06338634 -0.01607501 -0.01034895 -0.04650102  0.03723156
  0.00591584  0.07174399  0.01666446  0.06090763  0.06655257  0.02370512
  0.00057648  0.05684323  0.02416169 -0.00533628  0.04704772  0.01921572
  0.07682554  0.0056696  -0.07528222 -0.01713724 -0.07502711  0.07637348
 -0.0543796  -0.00138911 -0.01830185 -0.0467204  -0.04724139  0.02706785
  0.0323334   0.05537061  0.03370955 -0.01370663  0.00552706 -0.08226925
  0.01419507  0.06827912  0.01832052 -0.02147872  0.04149668 -0.020274
 -0.00601058  0.02448247 -0.0884009  -0.02566542 -0.03832614 -0.05610625
  0.04681281  0.03203127], shape=(50,), dtype=float32)


In [ ]:
print(embeddings.shape)
print(embeddings[0][:50])

(2, 512)
tf.Tensor(
[-0.03133019 -0.06338634 -0.01607501 -0.01034895 -0.04650102  0.03723156
  0.00591584  0.07174399  0.01666446  0.06090763  0.06655257  0.02370512
  0.00057648  0.05684323  0.02416169 -0.00533628  0.04704772  0.01921572
  0.07682554  0.0056696  -0.07528222 -0.01713724 -0.07502711  0.07637348
 -0.0543796  -0.00138911 -0.01830185 -0.0467204  -0.04724139  0.02706785
  0.0323334   0.05537061  0.03370955 -0.01370663  0.00552706 -0.08226925
  0.01419507  0.06827912  0.01832052 -0.02147872  0.04149668 -0.020274
 -0.00601058  0.02448247 -0.0884009  -0.02566542 -0.03832614 -0.05610625
  0.04681281  0.03203127], shape=(50,), dtype=float32)


In [ ]:
sentence_encoding_layer=hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
                                  input_shape=[],dtype=tf.string,trainable=False,name='USE')

In [ ]:
model_6=keras.Sequential([
    sentence_encoding_layer,
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid')
],name='model_6_USE')

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fa814105930> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)